### Procurando base de dados da steam
https://nik-davis.github.io/posts/2019/steam-data-collection/
https://steamspy.com/api.php

## Objetivo

Realize a extração das informações que conseguir da base de dados listada no website:

https://steamdb.info/sales/


Armazene estes dados no Google BigQuery

Em seguida exporte ou conecte esses dados em um Google Sheets e nos envie o link.

Atenção:

Você deve criar um repositório público em um GIT para compartilhar conosco;

Compartilhar o Sheets final (o link precisa ser público);

Lembrar de pôr no repositório os arquivos da automação;

In [ ]:
!pip install gspread

### Importando bibliotecas

In [ ]:
import datetime as dt
import time
import numpy as np
import pandas as pd
import requests


### Declaracao de funcao para processar requisicao

In [188]:
def get_request(url, parameters=None):
    try:
        response = requests.get(url=url, params=parameters)
    except SSLError as s:
        print('SSL Error:', s)
        
        for i in range(5, 0, -1):
            print('\rWaiting... ({})'.format(i), end='')
            time.sleep(1)
        print('\rRetrying.' + ' '*10)
        
        # recusively try again
        return get_request(url, parameters)
    
    if response:
        return response.json()
    else:
        print('No response, waiting 10 seconds...')
        time.sleep(10)
        print('Retrying.')
        return get_request(url, parameters)

### Listagem de atributos API (https://steamspy.com/api.php)
'appid', 'name', 'developer', 'publisher', 'score_rank', 'positive','negative', 'userscore', 'owners', 'average_forever', 'average_2weeks',
'median_forever', 'median_2weeks', 'price', 'initialprice', 'discount','languages', 'genre', 'ccu', 'tags'

### Extracao raw salva cvs para redundancia

In [216]:
url = "https://steamspy.com/api.php"
parameters = {"request": "all"}

json_data = get_request(url, parameters=parameters)
steam_spy_all = pd.DataFrame.from_dict(json_data, orient='index')

app_list = steam_spy_all[['appid', 'name','developer','discount','price','initialprice','negative','positive','score_rank']].sort_values('appid').reset_index(drop=True)

app_list.to_csv('app_list.csv', index=False)

In [190]:
app_list.head(20)

,appid,name,developer,discount,price,initialprice,negative,positive,score_rank
0,10,Counter-Strike,Valve,0,999,999,6217,235722,
1,20,Team Fortress Classic,Valve,0,499,499,1095,7324,
2,30,Day of Defeat,Valve,0,499,499,672,6262,
3,40,Deathmatch Classic,Valve,0,499,499,527,2550,
4,50,Half-Life: Opposing Force,Gearbox Software,0,499,499,1115,22312,
5,60,Ricochet,Valve,0,499,499,1011,4800,
6,70,Half-Life,Valve,0,999,999,4670,129718,
7,80,Counter-Strike: Condition Zero,Valve,0,999,999,2220,23356,
8,130,Half-Life: Blue Shift,Gearbox Software,0,499,499,1331,15910,
9,220,Half-Life 2,Valve,0,999,999,4858,196568,


### Carregamento dos dados em dataframe

In [191]:
steam_raw = pd.read_csv('app_list.csv')
steam_raw

,appid,name,developer,discount,price,initialprice,negative,positive,score_rank
0,10,Counter-Strike,Valve,0,999,999,6217,235722,NaN
1,20,Team Fortress Classic,Valve,0,499,499,1095,7324,NaN
2,30,Day of Defeat,Valve,0,499,499,672,6262,NaN
3,40,Deathmatch Classic,Valve,0,499,499,527,2550,NaN
4,50,Half-Life: Opposing Force,Gearbox Software,0,499,499,1115,22312,NaN
...,...,...,...,...,...,...,...,...,...
995,2567870,Chained Together,Anegar Games,0,499,499,3759,30193,NaN
996,2670630,Supermarket Simulator,Nokta Games,30,909,1299,3551,49800,NaN
997,2835570,Buckshot Roulette,Mike Klubnika,0,299,299,2032,36392,NaN
998,2881650,Content Warning,"thePetHen, Skog, Zorro, Wilnyl, Philip",0,799,799,5879,99916,NaN


### Verificacao dos tipos

In [192]:
print(steam_raw.dtypes)

appid             int64
name             object
developer        object
discount          int64
price             int64
initialprice      int64
negative          int64
positive          int64
score_rank      float64
dtype: object


In [193]:
steam_raw.describe()

,appid,discount,price,initialprice,negative,positive,score_rank
count,1.000000e+03,1000.000000,1000.000000,1000.000000,1.000000e+03,1.000000e+03,0.0
mean,5.548402e+05,7.994000,1662.394000,1891.226000,1.333821e+04,8.741777e+04,NaN
std,5.188649e+05,22.724092,1672.437731,1759.563576,5.468811e+04,2.798789e+05,NaN
min,1.000000e+01,0.000000,0.000000,0.000000,1.000000e+00,0.000000e+00,NaN
25%,2.226375e+05,0.000000,0.000000,0.000000,2.010750e+03,1.470200e+04,NaN
50%,3.795750e+05,0.000000,1499.000000,1999.000000,3.943500e+03,3.219750e+04,NaN
75%,7.759808e+05,0.000000,2499.000000,2999.000000,8.910750e+03,7.401775e+04,NaN
max,3.097560e+06,92.000000,9999.000000,9999.000000,1.093321e+06,7.285943e+06,NaN


### Ajuste de formatacao

In [194]:
steam_raw_formated = steam_raw.copy()

In [195]:
import requests
def pegar_cotacao(moeda_origem,moeda_destino):
    link= f"https://economia.awesomeapi.com.br/json/last/{moeda_origem}-{moeda_destino}"
    requisicao =requests.get(link)
    dic_resposta =requisicao.json()
    cotacao =dic_resposta[f"{moeda_origem}{moeda_destino}"]["bid"]
    cotacao_round = round(float(cotacao),2)
    return cotacao_round

print(f"Cotacao atual do dolar = R$ {pegar_cotacao('USD','BRL')}")

Cotacao atual do dolar = R$ 5.69


In [196]:
steam_raw_formated['discount'] = (1-(steam_raw_formated['price']/steam_raw_formated['initialprice']).round(2))*100
steam_raw_formated['total_reviews'] = steam_raw_formated['negative'] + steam_raw_formated['positive']
steam_raw_formated['score_rank'] = ((steam_raw_formated['positive']/steam_raw_formated['total_reviews']).round(2))*100
steam_raw_formated['price'] = (steam_raw_formated['price']/100 * pegar_cotacao("USD","BRL")).round(2)
steam_raw_formated['initialprice'] = (steam_raw_formated['initialprice']/100 * pegar_cotacao("USD","BRL")).round(2)

steam_raw_formated

,appid,name,developer,discount,price,initialprice,negative,positive,score_rank,total_reviews
0,10,Counter-Strike,Valve,0.0,56.84,56.84,6217,235722,97.0,241939
1,20,Team Fortress Classic,Valve,0.0,28.39,28.39,1095,7324,87.0,8419
2,30,Day of Defeat,Valve,0.0,28.39,28.39,672,6262,90.0,6934
3,40,Deathmatch Classic,Valve,0.0,28.39,28.39,527,2550,83.0,3077
4,50,Half-Life: Opposing Force,Gearbox Software,0.0,28.39,28.39,1115,22312,95.0,23427
...,...,...,...,...,...,...,...,...,...,...
995,2567870,Chained Together,Anegar Games,0.0,28.39,28.39,3759,30193,89.0,33952
996,2670630,Supermarket Simulator,Nokta Games,30.0,51.72,73.91,3551,49800,93.0,53351
997,2835570,Buckshot Roulette,Mike Klubnika,0.0,17.01,17.01,2032,36392,95.0,38424
998,2881650,Content Warning,"thePetHen, Skog, Zorro, Wilnyl, Philip",0.0,45.46,45.46,5879,99916,94.0,105795


### Limpeza de Nulos

In [197]:
steam_raw_formated[steam_raw_formated['discount'].isna()]

,appid,name,developer,discount,price,initialprice,negative,positive,score_rank,total_reviews
13,340,Half-Life 2: Lost Coast,Valve,NaN,0.0,0.0,1471,12220,89.0,13691
17,440,Team Fortress 2,Valve,NaN,0.0,0.0,129474,993709,88.0,1123183
20,570,Dota 2,Valve,NaN,0.0,0.0,432170,1915977,82.0,2348147
22,630,Alien Swarm,Valve,NaN,0.0,0.0,1088,20030,95.0,21118
23,730,Counter-Strike: Global Offensive,Valve,NaN,0.0,0.0,1093321,7285943,87.0,8379264
...,...,...,...,...,...,...,...,...,...,...
960,1677740,Stumble Guys,Scopely,NaN,0.0,0.0,18020,103120,85.0,121140
963,1721470,Poppy Playtime,Mob Entertainment,NaN,0.0,0.0,13179,65882,83.0,79061
967,1811260,EA SPORTS FIFA 23,EA Canada &amp; EA Romania,NaN,0.0,0.0,65406,85913,57.0,151319
969,1818750,MultiVersus,Player First Games,NaN,0.0,0.0,26423,90492,77.0,116915


In [198]:
res_missing = steam_raw_formated.isna().sum()
res_missing = (res_missing/len(steam_raw_formated))*100
res_missing.sort_values(ascending=False)

discount         25.2
appid             0.0
name              0.0
developer         0.0
price             0.0
initialprice      0.0
negative          0.0
positive          0.0
score_rank        0.0
total_reviews     0.0
dtype: float64

In [199]:
steam_raw_formated.dropna(subset=["discount"], inplace=True)

In [200]:
steam_raw_formated.describe()

,appid,discount,price,initialprice,negative,positive,score_rank,total_reviews
count,7.480000e+02,748.000000,748.000000,748.000000,748.000000,7.480000e+02,748.000000,7.480000e+02
mean,5.571811e+05,10.687166,126.454693,143.861591,10044.969251,8.533475e+04,87.316845,9.537972e+04
std,5.436349e+05,25.724802,89.863270,90.458570,23401.235690,1.554235e+05,10.323801,1.711492e+05
min,1.000000e+01,0.000000,2.790000,5.630000,1.000000,0.000000e+00,0.000000,1.000000e+00
25%,2.197150e+05,0.000000,56.840000,85.290000,1921.250000,1.721650e+04,83.000000,2.006525e+04
50%,3.695850e+05,0.000000,113.740000,113.740000,3892.500000,3.729650e+04,90.000000,4.338150e+04
75%,7.744860e+05,0.000000,170.640000,170.640000,8570.750000,8.265725e+04,94.000000,9.694150e+04
max,3.097560e+06,92.000000,568.940000,568.940000,265333.000000,1.651029e+06,99.000000,1.897135e+06


### Definicao de dataframe final par exportacao

In [201]:
steam_bronze = steam_raw_formated.copy()

In [202]:
steam_bronze.drop(columns=['initialprice','negative','positive','total_reviews','developer'],inplace=True)


In [203]:
print(steam_bronze.dtypes)

appid           int64
name           object
discount      float64
price         float64
score_rank    float64
dtype: object


In [206]:
steam_bronze.sort_values(by=["discount", "name"], ascending=[False, True], inplace=True)
steam_bronze = steam_bronze[steam_bronze["discount"] != 0]
steam_bronze

,appid,name,discount,price,score_rank
940,1517290,Battlefield 2042,92.0,27.26,45.0
658,582660,Black Desert,90.0,5.63,74.0
701,645630,Car Mechanic Simulator 2018,90.0,11.32,91.0
339,261110,Killer is Dead - Nightmare Edition,90.0,11.32,75.0
471,351940,The Descendant,90.0,8.48,89.0
...,...,...,...,...,...
828,1086940,Baldur's Gate 3,20.0,273.06,97.0
864,1203620,Enshrouded,20.0,136.50,86.0
767,838350,太吾绘卷 The Scroll Of Taiwu,15.0,96.67,69.0
992,2379780,Balatro,10.0,76.76,97.0


### Transformacao em xls -> Google docs

In [205]:
steam_bronze.to_excel("steam_bronze_formated.xlsx", index=False)